In [1]:
import os
import random
import tqdm

import jax
import jax.numpy as jnp
from jax import grad, jit, vmap
import flax.linen as nn
import optax
import torch


import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from Bio import Entrez, SeqIO

In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM


# Télécharger le modèle de langage d'ADN et le tokenizer.
tokenizer = AutoTokenizer.from_pretrained(
    "InstaDeepAI/nucleotide-transformer-v2-50m-multi-species",
    trust_remote_code=True)

language_model = AutoModelForMaskedLM.from_pretrained(
    "InstaDeepAI/nucleotide-transformer-v2-50m-multi-species",
    trust_remote_code=True)

d:\master\internship\project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# @title Vérifier TPU/GPU. (Exécuter la cellule)
import jax
num_devices = jax.device_count()
device_type = jax.devices()[0].device_kind
print(f"Trouvé {num_devices} dispositif(s) JAX de type {device_type}.")

Trouvé 1 dispositif(s) JAX de type cpu.


In [4]:
from Bio import SeqIO
sequences = []
for record in SeqIO.parse("data/raw/Gossypium_hirsutum_v2.1_genomic.fna", "fasta"):
    sequences.append(record.seq)

print(f"Nombre de séquences chargées : {len(sequences)}")

Nombre de séquences chargées : 1027


In [5]:
bt_sequences = []
for record in SeqIO.parse("data/raw/bt_genes.fasta", "fasta"):
    bt_sequences.append(record.seq)

In [6]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import random

synthetic_sequences = []

for cotton in sequences[:10]:
    bt = random.choice(bt_sequences)
    combined = cotton[:500] + bt[:300]
    
    synthetic_sequences.append(
        SeqRecord(
            Seq(combined),
            id="MON15985_like",
            description="Synthetic GMO cotton"
        )
    )

SeqIO.write(synthetic_sequences, "data/processed/gmo_cotton_MON15985_like.fasta", "fasta")


10

Build training data

In [7]:
from datasets import Dataset

data = []

# Natural cotton
for s in sequences[:20]:
    data.append({
        "sequence": str(s),
        "label": 0
    })

# GMO-like cotton
for s in synthetic_sequences:
    data.append({
        "sequence": str(s),
        "label": 1
    })

dataset = Dataset.from_list(data)

dataset = dataset.map(
    lambda x: tokenize(x["sequence"]),
    batched=True
)


KeyboardInterrupt: 

FineTune model